In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import os
from prettytable import PrettyTable
from pylab import *
from scipy.stats import wasserstein_distance
import random
import csv
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

device = "cuda" if torch.cuda.is_available() else "cpu"

### Move Working Directory

In [2]:
# MaryClare's
#os.chdir('/Users/maryclaremartin/Documents/jup/ExtraSensory')

# Josh's
#os.chdir("D:/Research_Projects/REU2021-human-context-recognition/UCI_data")

# Harrison's
os.chdir("/Users/hkimr/Desktop/WPI Github/REU2021-human-context-recognition/UCI_data")

#os.chdir('../')
softmax = nn.Softmax(dim = 1)

### Load & Scale Data

In [3]:
##load and scale data
#returns scaled data (X) and labels (Y)
#label: array, list of activities to use (represented by integers)
#users: array, list of users whose data is to be used (represented by integers)

def start_data(label, users):
    #get the dataframe column names
    name_dataframe = pd.read_csv('data/features.txt', delimiter = '\n', header = None)
    names = name_dataframe.values.tolist()
    names = [k for row in names for k in row] #List of column names

    data = pd.read_csv('data/X_train.txt', delim_whitespace = True, header = None) #Read in dataframe
    data.columns = names #Setting column names

    #X_train = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z'] #Selecting only acceleration columns
    
    X_train_1 = data.loc[:,'1 tBodyAcc-mean()-X':'40 tBodyAcc-correlation()-Y,Z']
    X_train_2 = data.loc[:,'81 tBodyAccJerk-mean()-X':'120 tBodyAccJerk-correlation()-Y,Z']
    X_train = pd.concat([X_train_1, X_train_2], axis = 1)
    
    y_train_activity = pd.read_csv('data/y_train.txt', header = None)
    y_train_activity.columns = ['Activity']

    y_train_subject = pd.read_csv('data/subject_train.txt', header = None)
    y_train_subject.columns = ['Subject']

    GAN_data = pd.concat([X_train, y_train_activity, y_train_subject], axis = 1)
    GAN_data = GAN_data[GAN_data['Activity'].isin(label)]
    GAN_data = GAN_data[GAN_data['Subject'].isin(users)]
    
    X = GAN_data.loc[:,'1 tBodyAcc-mean()-X':'120 tBodyAccJerk-correlation()-Y,Z'].values
    y = GAN_data[['Activity']].values
    
    return X, y

### The Generator

In [4]:
#defines each generator layer
#input and output dimensions needed
def generator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace = True)
    )

#returns n_samples of z_dim (number of dimensions of latent space) noise
def get_noise(n_samples, z_dim):
    return torch.randn(n_samples, z_dim).to(device)

#defines generator class
class Generator(nn.Module):
    def __init__(self, z_dim = 10, feature_dim = 80, hidden_dim = 128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_block(z_dim, 95),
            generator_block(95, 90),
            generator_block(90, 85),
            nn.Linear(85, feature_dim)
        )
    def forward(self, noise):
        return self.gen(noise)

##calculates generator loss
#gen: generator
#disc: discriminator
#criterion1: loss function1
#criterion2: loss function2
#batch_size: batch size
#z_dim: number of dimensions in the latent space
def get_gen_loss(gen, disc, act, usr, criterion1, criterion2, batch_size, z_dim, activities, users):
    latent_vectors = get_noise(batch_size, z_dim)
    act_vectors = get_act_matrix(batch_size, activities)
    usr_vectors = get_usr_matrix(batch_size, users)
    
    to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
    fake_features = gen(to_gen)
    
    disc.eval()
    pred_disc = disc(fake_features)
    disc.train()
    pred_act = act(fake_features) ### CrossEntropyLoss Criterion automatically applies softmax and torch.max
    pred_usr = usr(fake_features)
    
    d_loss = criterion1(pred_disc, torch.ones_like(pred_disc))
    act_loss = criterion2(pred_act, act_vectors[0])
    usr_loss = criterion2(pred_usr, usr_vectors[0])
    
    gen_loss = d_loss + act_loss + usr_loss
    return gen_loss
    
def get_act_matrix(batch_size, a_dim):
    indexes = np.random.randint(a_dim, size = batch_size)
    
    one_hot = np.zeros((len(indexes), indexes.max()+1))
    one_hot[np.arange(len(indexes)),indexes] = 1
    return torch.Tensor(indexes).long().to(device), torch.Tensor(one_hot).to(device)
    
def get_usr_matrix(batch_size, u_dim):
    indexes = np.random.randint(u_dim, size = batch_size)
    
    one_hot = np.zeros((indexes.size, indexes.max()+1))
    one_hot[np.arange(indexes.size),indexes] = 1
    return torch.Tensor(indexes).long().to(device), torch.Tensor(one_hot).to(device)

In [5]:
# #defines each generator layer
# #input and output dimensions needed
# def generator_block(input_dim, output_dim):
#     return nn.Sequential(
#         nn.Linear(input_dim, output_dim),
#         nn.Dropout(0.1),
#         nn.BatchNorm1d(output_dim),
#         nn.ReLU(inplace = True)
#     )

# #returns n_samples of z_dim (number of dimensions of latent space) noise
# def get_noise(n_samples, z_dim):
#     return torch.randn(n_samples, z_dim).to(device)

# #defines generator class
# class Generator(nn.Module):
#     def __init__(self, z_dim = 10, feature_dim = 40, hidden_dim = 128):
#         super(Generator, self).__init__()
#         self.gen = nn.Sequential(
#             generator_block(z_dim, int(hidden_dim/2)),
#             generator_block(int(hidden_dim/2), 50),
#             generator_block(50, 45),
#             nn.Linear(45, feature_dim)
#         )
#     def forward(self, noise):
#         return self.gen(noise)

# ##calculates generator loss
# #gen: generator
# #disc: discriminator
# #criterion1: loss function1
# #criterion2: loss function2
# #batch_size: batch size
# #z_dim: number of dimensions in the latent space
# def get_gen_loss(gen, disc, act, usr, criterion1, criterion2, batch_size, z_dim, activities, users):
#     latent_vectors = get_noise(batch_size, z_dim)
#     act_vectors = get_act_matrix(batch_size, activities)
#     usr_vectors = get_usr_matrix(batch_size, users)
    
#     to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
#     fake_features = gen(to_gen)
    
#     disc.eval()
#     pred_disc = disc(fake_features)
#     disc.train()
#     pred_act = act(fake_features) ### CrossEntropyLoss Criterion automatically applies softmax and torch.max
#     pred_usr = usr(fake_features)
    
#     d_loss = criterion1(pred_disc, torch.ones_like(pred_disc))
#     act_loss = criterion2(pred_act, act_vectors[0])
#     usr_loss = criterion2(pred_usr, usr_vectors[0])
    
#     gen_loss = d_loss + act_loss + usr_loss
#     return gen_loss
    
# def get_act_matrix(batch_size, a_dim):
#     indexes = np.random.randint(a_dim, size = batch_size)
    
#     one_hot = np.zeros((len(indexes), indexes.max()+1))
#     one_hot[np.arange(len(indexes)),indexes] = 1
#     return torch.Tensor(indexes).long().to(device), torch.Tensor(one_hot).to(device)
    
# def get_usr_matrix(batch_size, u_dim):
#     indexes = np.random.randint(u_dim, size = batch_size)
    
#     one_hot = np.zeros((indexes.size, indexes.max()+1))
#     one_hot[np.arange(indexes.size),indexes] = 1
#     return torch.Tensor(indexes).long().to(device), torch.Tensor(one_hot).to(device)

### Create Fake Generated Samples

In [6]:
def get_fake_samples(gen, batch_size, z_dim):
    """
    Generates fake acceleration features given a batch size, latent vector dimension, and trained generator.
    
    """
    latent_vectors = get_noise(batch_size, z_dim) ### Retrieves a 2D tensor of noise
    gen.eval()
    fake_features = gen(latent_vectors)
    gen.train()
    return fake_features ### Returns a 2D tensor of fake features of size batch_size x z_dim

### The Discriminator

In [7]:
def discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

#defines discriminator class
class Discriminator(nn.Module):
    def __init__(self, feature_dim = 80, hidden_dim = 16):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            discriminator_block(feature_dim, 60),
            discriminator_block(60, 30),
            nn.Linear(30, 1),
            nn.Sigmoid()
        )
    def forward(self, feature_vector):
        return self.disc(feature_vector)
    
def get_disc_loss(gen, disc, criterion, real_features, batch_size, z_dim, a_dim, u_dim):
    latent_vectors = get_noise(batch_size, z_dim)
    act_vectors = get_act_matrix(batch_size, a_dim)
    usr_vectors = get_usr_matrix(batch_size, u_dim)
    
    to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
    gen.eval()
    fake_features = gen(to_gen)
    gen.train()
    pred_fake = disc(fake_features.detach())
    
    ground_truth = torch.zeros_like(pred_fake)
    loss_fake = criterion(pred_fake, ground_truth)
    
    pred_real = disc(real_features)
    ground_truth = torch.ones_like(pred_real)
    loss_real = criterion(pred_real, ground_truth)
    
    disc_loss = (loss_fake + loss_real) / 2
    return disc_loss

In [8]:
# def discriminator_block(input_dim, output_dim):
#     return nn.Sequential(
#         nn.Linear(input_dim, output_dim),
#         nn.Dropout(0.1),
#         nn.LeakyReLU(0.05)
#     )

# #defines discriminator class
# class Discriminator(nn.Module):
#     def __init__(self, feature_dim = 40, hidden_dim = 16):
#         super(Discriminator, self).__init__()
#         self.disc = nn.Sequential(
#             discriminator_block(feature_dim, hidden_dim),
#             discriminator_block(hidden_dim, int(hidden_dim/2)),
#             nn.Linear(int(hidden_dim/2), 1),
#             nn.Sigmoid()
#         )
#     def forward(self, feature_vector):
#         return self.disc(feature_vector)
    
# def get_disc_loss(gen, disc, criterion, real_features, batch_size, z_dim, a_dim, u_dim):
#     latent_vectors = get_noise(batch_size, z_dim)
#     act_vectors = get_act_matrix(batch_size, a_dim)
#     usr_vectors = get_usr_matrix(batch_size, u_dim)
    
#     to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
#     gen.eval()
#     fake_features = gen(to_gen)
#     gen.train()
#     pred_fake = disc(fake_features.detach())
    
#     ground_truth = torch.zeros_like(pred_fake)
#     loss_fake = criterion(pred_fake, ground_truth)
    
#     pred_real = disc(real_features)
#     ground_truth = torch.ones_like(pred_real)
#     loss_real = criterion(pred_real, ground_truth)
    
#     disc_loss = (loss_fake + loss_real) / 2
#     return disc_loss


### User Classifier

In [9]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class User_Classifier(nn.Module):
    def __init__(self, feature_dim = 80):
        super(User_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 60),
            classifier_block(60, 40),
            classifier_block(40, 20),
            classifier_block(20, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

In [10]:
# def classifier_block(input_dim, output_dim):
#     return nn.Sequential(
#         nn.Linear(input_dim, output_dim),
#         nn.Dropout(0.1),
#         nn.LeakyReLU(0.05)
#     )

# class User_Classifier(nn.Module):
#     def __init__(self, feature_dim = 40):
#         super(User_Classifier, self).__init__()
#         self.network = nn.Sequential(
#             classifier_block(feature_dim, 30),
#             classifier_block(30, 25),
#             classifier_block(25, 20),
#             classifier_block(20, 10),
#             nn.Linear(10, 3)
#         )
#     def forward(self, x):
#         return self.network(x)

### Activity Classifier

In [11]:
def classifier_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.Dropout(0.1),
        nn.LeakyReLU(0.05)
    )

class Activity_Classifier(nn.Module):
    def __init__(self, feature_dim = 80):
        super(Activity_Classifier, self).__init__()
        self.network = nn.Sequential(
            classifier_block(feature_dim, 60),
            classifier_block(60, 40),
            classifier_block(40, 20),
            classifier_block(20, 10),
            nn.Linear(10, 3)
        )
    def forward(self, x):
        return self.network(x)

In [12]:
# def classifier_block(input_dim, output_dim):
#     return nn.Sequential(
#         nn.Linear(input_dim, output_dim),
#         nn.Dropout(0.1),
#         nn.LeakyReLU(0.05)
#     )

# class Activity_Classifier(nn.Module):
#     def __init__(self, feature_dim = 40):
#         super(Activity_Classifier, self).__init__()
#         self.network = nn.Sequential(
#             classifier_block(feature_dim, 30),
#             classifier_block(30, 25),
#             classifier_block(25, 20),
#             classifier_block(20, 10),
#             nn.Linear(10, 3)
#         )
#     def forward(self, x):
#         return self.network(x)

### Calculate Performance Statistics

In [13]:
##calculates performance statistics for each epoch of training
#gen: generator
#disc: discriminator
#b_size: batch size
#z_dim: number of dimensions of the latent space
##returns accuracy, precision, recall, fpR, and f1 score
def performance_stats(gen, disc, b_size, z_dim, a_dim, u_dim, batch = None):
    tp = 0 #true positive
    fp = 0 #false positive
    tn = 0 #true negative
    fn = 0 #false negative

    with torch.no_grad():
        y_hat_fake = torch.Tensor([0])
        y_hat_real = torch.Tensor([0])
        
        if batch is None:
            latent_vectors = get_noise(b_size, z_dim)
            act_vectors = get_act_matrix(b_size, a_dim)
            usr_vectors = get_usr_matrix(b_size, u_dim)
            to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
            gen.eval()
            fake_features = gen(to_gen)
            gen.train()
            disc.eval()
            y_hat_fake = torch.round(disc(fake_features))
            disc.train()
        else:
            latent_vectors = get_noise(b_size, z_dim)
            act_vectors = get_act_matrix(b_size, a_dim)
            usr_vectors = get_usr_matrix(b_size, u_dim)
            to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
            gen.eval()
            fake_features = gen(to_gen)
            gen.train()
            disc.eval()
            y_hat_fake = torch.round(disc(fake_features))
            y_hat_real = torch.round(disc(batch))
            disc.train()
         
        fpR = torch.mean(y_hat_fake)
        recall = torch.mean(y_hat_real)
        class_acc = ((len(y_hat_fake) - torch.sum(y_hat_fake)) + torch.sum(y_hat_real)) / (2*b_size)
        #print(f'Classification Accuracy: {class_acc:.2f}'
        #print(f'Recall: {recall:.2f}') #What percent of the true positives were identified
        return class_acc, recall, fpR
      

def performance_stats_class(gen, a_class, u_class, batch_size, z_dim, a_dim, u_dim):
    tp = 0 #true positive
    fp = 0 #false positive
    tn = 0 #true negative
    fn = 0 #false negative

    with torch.no_grad():
        latent_vectors = get_noise(batch_size, z_dim)
        act_vectors = get_act_matrix(batch_size, a_dim)
        usr_vectors = get_usr_matrix(batch_size, u_dim)
    
        to_gen = torch.cat((latent_vectors, act_vectors[1], usr_vectors[1]), 1)
        gen.eval()
        fake_features = gen(to_gen)
        gen.train()
    
        _, pred_a_class = torch.max(softmax(a_class(fake_features)), dim = 1)
        _, pred_u_class = torch.max(softmax(u_class(fake_features)), dim = 1)
        
        return torch.eq(act_vectors[0], pred_a_class).sum()/batch_size, torch.eq(usr_vectors[0], pred_u_class).sum()/batch_size

### Initialize Training Environment

In [14]:
###initalize parameters that depend on training loop parameters
#X: acceleration data
#y: labels associated with X data (fake or real)
#z_dim: number of dimensions to the latent space
#disc_lr: discriminator learning rate
#gen_lr: generator learning rate
#DISCRIMINATOR: 1 to indicate if discriminator is training
#batch_size: batch size
#disc: initialized discrimiantor

def initialize_params(X, y, z_dim, a_dim, u_dim, disc_lr, gen_lr, DISCRIMINATOR, batch_size, disc):
    #initialize generator
    gen = Generator(z_dim + a_dim + u_dim).to(device)
    #indicate that discriminator is training
    to_train = DISCRIMINATOR
    #create training features
    train_features = torch.tensor(X)
    #create training labels
    train_labels = torch.tensor(y)
    #concatenate to create training data
    train_data = torch.utils.data.TensorDataset(train_features, train_labels)
    #create data loader for training data
    train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, shuffle = True)
    #initialize generator and discriminator optimizers
    opt_disc = optim.Adam(disc.parameters(), lr = disc_lr)
    opt_gen = optim.Adam(gen.parameters(), lr = gen_lr)
    
    return gen, to_train, train_features, train_labels, train_data, train_loader, opt_disc, opt_gen   

# Save / Load Models

In [15]:
# Change path and name of the Generator and Discriminator accordingly
def save_model(gen, disc, model_name):
    torch.save(gen.state_dict(), f"saved_models/{model_name}_gen.param")
    torch.save(disc.state_dict(), f"saved_models/{model_name}_disc.param")
    
def load_model(model, model_name):
    model.load_state_dict(torch.load(f'saved_models/{model_name}'))

# Training

In [16]:
######Training loop to train GAN

#Parameters to specifiy: 
    #X: starting accelerometer data
    #y: starting labels for X data (fake or real)
    
#Set parameters (do not change)
    #criterion: loss function (BCE)
    #dig: number of significant digits for printing (5)
    #feature_dim: Number of dimensions of output from generator (26)
    #GENERATOR: set generator to zero for training
    #DISCRIMINATOR: set discriminator to one for training
    #train_string: starting machine to train (DISC)
    #disc: initalize discriminator
    #rel_epochs: Epochs passed since last switch (constant training) (0)
    #rows: initialization of array to save data of each epoch to CSV file ([])
    #heading: array of column headings for table (["Epoch", "Machine Training", "Discriminator Loss", 
                    #"Generator Loss", "FPR", "Recall", "Median Wasserstein", "Mean Wasserstein"])
    #table: intialize a table as a pretty table to save epoch data
    #switch_count: number of switches in dynamic training (0)
    
#Set parameters (can change):
    #z_dim: number of dimensions of latent vector (100)
    #gen_lr: generator learning rate (.001)
    #disc_lr: discriminator learning rate (.001) (shoud be equal to gen_lr)
    #batch_size: batch size (75)
    #print_batches: Show model performance per batch (False)
    #n_epochs: number of epochs to train (100)
    #constant_train_flag: (False)
        #Set to true to train based on constant # of epochs per machine 
        #Set to false to train dynamically based on machine performance
        
    #Constant training approach:
        #disc_epochs: Number of consecutive epochs to train discriminator before epoch threshold (5)
        #gen_epochs: Number of consecutive epochs to train generator before epoch threshold (2)
        #epoch_threshold: Epoch number to change training epoch ratio (50)
        #disc_epochs_change: New number of consecutive epochs to train discriminator after epoch threshold is exceeded (1)
        #gen_epochs_change: New number of consecutive epochs to train generator after epoch threshold is exceeded (50)
    
    #Dynamic training approach:                        
        #static_threshold: Epoch number to change from static ratio to dynamic (18)
        #static_disc_epochs: Number of consecutive epochs to train discriminator before epoch threshold (4)
        #static_gen_epochs: Number of consecutive epochs to train generator before epoch threshold (2)
        #pull_threshold: Accuracy threshold for switching machine training when the generator is no longer competitive (0.4)
        #push_threshold: Accuracy threshold for switching machine training when the discriminator is no longer competitive (0.6)
        #recall_threshold: threshold for recall to switch machine training when discriminator is training well
        #switch_flag: indicates if we should switch our training machine (False)
        
def training_loop(X, y, act, usr, criterion1 = nn.BCELoss(), criterion2 = nn.CrossEntropyLoss(), gan_id = "Mod Test Gan", dig = 5, feature_dim = 40, 
                  GENERATOR = 0, DISCRIMINATOR = 1, train_string = "DISC", disc = Discriminator(), z_dim = 100, a_dim = 3, u_dim = 3, 
                  gen_lr =  0.001, disc_lr = 0.001, batch_size = 100, constant_train_flag = False, disc_epochs = 5,
                  gen_epochs = 2, epoch_threshold = 50, disc_epochs_change = 5, gen_epochs_change = 2, rel_epochs = 0,
                 static_threshold = 28, static_disc_epochs = 5, static_gen_epochs = 2, pull_threshold = 0.3,
                 push_threshold = 0.7, recall_threshold = 0.75, n_epochs = 1000, rows = [],
                 heading = ["Epoch", "Training", "Discriminator Loss", "Generator Loss", "D_Accuracy", "D_fpR", "D_Recall", "A_fpR", "U_fpR"],
                 table = PrettyTable(), switch_flag = False, switch_count = 0, last_real_features = []):
    
    disc.to(device)
    #returns generator, sets discriminator training, creates training tensor, loads data, and initializes optimizers
    gen, to_train, train_features, train_labels, train_data, train_loader, opt_disc, opt_gen = initialize_params(X, y, z_dim, a_dim, u_dim, disc_lr, gen_lr, DISCRIMINATOR, batch_size, disc)

    #set pretty table field names
    table.field_names = heading
    
    #visualize_gen(X, gen, z_dim, a_dim, u_dim)
    gen_epochs = 0
    
    last_D_loss = -1.0
    last_G_loss = -1.0
    
    mean_mean = []
    mean_median = []
    
    for epoch in range(n_epochs):  
        if constant_train_flag:
            if to_train == DISCRIMINATOR and rel_epochs >= disc_epochs:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

            elif to_train == GENERATOR and rel_epochs >= gen_epochs:
                rel_epochs = 0
                to_train = DISCRIMINATOR
                train_string = "DISC"

            # Change epoch ratio after intial 'leveling out'
            if epoch == epoch_threshold:
                rel_epochs = 0
                to_train = GENERATOR
                train_string = "GEN"

                old_ratio = gen_epochs / disc_epochs
                gen_epochs = gen_epochs_change
                disc_epochs = disc_epochs_change
                new_ratio = gen_epochs / disc_epochs
                print(f'\n\nTraining ratio of G/D switched from {old_ratio:.{dig}f} to {new_ratio:.{dig}f}\n\n')
        else:
            if epoch < static_threshold:
                if to_train == DISCRIMINATOR and rel_epochs >= static_disc_epochs:
                    rel_epochs = 0
                    to_train = GENERATOR
                    train_string = "GEN"

                elif to_train == GENERATOR and rel_epochs >= static_gen_epochs:
                    rel_epochs = 0
                    to_train = DISCRIMINATOR
                    train_string = "DISC"

            else:
                if not switch_flag:
                    print("\nSwitching to Dynamic Training\n")
                    switch_flag = True
                    to_train = DISCRIMINATOR
                    train_string = "DISC"
                if to_train == DISCRIMINATOR and fpR <= pull_threshold and R >= recall_threshold:
                    to_train = GENERATOR
                    train_string = "GEN"
                    print("\nPull Generator\n")
                    switch_count += 1
                if to_train == GENERATOR and fpR >= push_threshold:
                    to_train = DISCRIMINATOR
                    train_string = "DISC"
                    print("\nPush Generator\n")
                    switch_count += 1
                    
        print(f'Epoch[{epoch + 1}/{n_epochs}] Train: {train_string} ', end ='')
        
        for batch_idx, (real_features, _) in enumerate(train_loader):
            #Size of this current batch
            batch_size = len(real_features)
            #Send batch to GPU
            real_features = real_features.float().to(device)
            
            if to_train == DISCRIMINATOR:
                ### Training Discriminator
                #visualize_real_batch(real_features.float())
                opt_disc.zero_grad()
                disc_loss = get_disc_loss(gen, disc, criterion1, real_features, batch_size, z_dim, a_dim, u_dim)
                disc_loss.backward(retain_graph = True)
                opt_disc.step()
                last_D_loss = disc_loss.item()
                # If generator has yet to ever train, run intial evaluation
                if last_G_loss == -1.0:
                    last_G_loss = get_gen_loss(gen, disc, act, usr, criterion1, criterion2, batch_size, z_dim, a_dim, u_dim)
                
            else:
                ### Training Generator
                opt_gen.zero_grad()
                gen_loss = get_gen_loss(gen, disc, act, usr, criterion1, criterion2, batch_size, z_dim, a_dim, u_dim)
                gen_loss.backward()
                opt_gen.step()
                last_G_loss = gen_loss.item()
                # If discriminator has yet to ever train, run intial evaluation
                if last_D_loss == -1.0:
                    last_D_loss = get_disc_loss(gen, disc, criterion1, real_features, batch_size, z_dim, a_dim, u_dim)
                
            if batch_idx == (len(train_loader) - 1):
                acc, R, fpR = performance_stats(gen, disc, batch_size, z_dim, a_dim, u_dim, batch = real_features)
                A_fpR, U_fpR = performance_stats_class(gen, act, usr, batch_size, z_dim, a_dim, u_dim)
                #w_dist = all_Wasserstein_dists(gen, z_dim, feature_dim, real_features.float())
                #median_w_dist = torch.median(w_dist)
                #mean_w_dist = torch.mean(w_dist)
                
                #mean_mean.append(mean_w_dist)
                #mean_median.append(median_w_dist)
                
        mean_mean_w = torch.mean(torch.Tensor(mean_mean)) 
        mean_median_w = torch.mean(torch.Tensor(mean_median))
            
        ### Currently doesn't print Median/Mean Wasserstein --> Change if needed
        print(f'| LossD: {last_D_loss:.{dig}f}, LossG: {last_G_loss:.{dig}f} | Acc: {acc:.{dig}f} | fpR: {fpR:.{dig}f} | R: {R:.{dig}f} | A_fpR: {A_fpR:.{dig}f} | U_fpR: {U_fpR:.{dig}f}')
        row_to_add = [f"{epoch + 1}", train_string, f"{last_D_loss:.{dig}f}", f"{last_G_loss:.{dig}f}", f"{acc:.{dig}f}", f"{fpR:.{dig}f}", f"{R:.{dig}f}", f"{A_fpR:.{dig}f}", f"{U_fpR:.{dig}f}"]
        table.add_row(row_to_add)
        rows.append(row_to_add)
                
        if to_train == GENERATOR:
            gen_epochs += 1
            
        mean_mean.clear()
        mean_median.clear()
        rel_epochs += 1
        
        
    print("\n\nTraining Session Finished")
    print(f"Encountered {switch_count} non-trivial training swaps")
    percent = gen_epochs / n_epochs
    print(f"Trained Generator {gen_epochs} out of {n_epochs} ({percent:.3f})")
    f = open("model_outputs/" + gan_id + ".txt", "w")
    f.write(table.get_string())
    f.close()
    print("Model Results Sucessfully Saved to \"model_outputs/" + gan_id + ".txt\"")

    with open("model_outputs/" + gan_id + ".csv", "w") as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 
        # writing the fields 
        csvwriter.writerow(heading)
        # writing the data rows 
        csvwriter.writerows(rows)
    print("Model Results Sucessfully Saved to \"model_outputs/" + gan_id + ".csv\"")
    save_model(gen, disc, gan_id)
    print("Generator's Trained Parameters Sucessfully Saved to \"saved_models/" + gan_id + "_gen.param\"")
    print("Discriminators's Trained Parameters Sucessfully Saved to \"saved_models/" + gan_id + "_disc.param\"")
    model_output = pd.read_csv("model_outputs/" + gan_id + ".csv")
    #visualize_gen(X, gen, z_dim)
    # Change path and name of the Generator and Discriminator accordingly
    save_model(gen, disc, gan_id)
    
    return model_output

# Run Training

In [17]:
#X, y = start_data("aggregated_data/aggregated_data.csv", "label:SITTING")
#X, y = start_data("raw_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv", "label:SITTING" ) 
X, y = start_data([1, 3, 4], [1, 3, 5])

#Initialize Classifier Architectures
activity_classifier = Activity_Classifier()
user_classifier = User_Classifier()

#Freeze Them
activity_classifier.eval()
user_classifier.eval()
#Send to GPU
activity_classifier.to(device)
user_classifier.to(device)

#Load with pre-trained parameters
activity_classifier.load_state_dict(torch.load('saved_models/UCI 3 Label Classifier'))
user_classifier.load_state_dict(torch.load("saved_models/UCI 3 User Classifier"))

#Train
model_output = training_loop(X,y, activity_classifier, user_classifier, gan_id="cGAN_UCI", batch_size = len(X), gen_lr=.0001, disc_lr =.0001, n_epochs=20000, dig=5, constant_train_flag=False)

Epoch[1/20000] Train: DISC | LossD: 0.70212, LossG: 130.00882 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.34809 | U_fpR: 0.29376
Epoch[2/20000] Train: DISC | LossD: 0.70160, LossG: 130.00882 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.32596 | U_fpR: 0.33803
Epoch[3/20000] Train: DISC | LossD: 0.70090, LossG: 130.00882 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.32797 | U_fpR: 0.35614
Epoch[4/20000] Train: DISC | LossD: 0.70014, LossG: 130.00882 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.32394 | U_fpR: 0.35412
Epoch[5/20000] Train: DISC | LossD: 0.69946, LossG: 130.00882 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.34809 | U_fpR: 0.31791
Epoch[6/20000] Train: GEN | LossD: 0.69946, LossG: 133.10414 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.34004 | U_fpR: 0.33602
Epoch[7/20000] Train: GEN | LossD: 0.69946, LossG: 127.43967 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.32193 | U_fpR: 0.32797
Epoch[8/20000] Train: DISC | 

Epoch[66/20000] Train: DISC | LossD: 0.64884, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.31187 | U_fpR: 0.32797
Epoch[67/20000] Train: DISC | LossD: 0.64841, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.39638 | U_fpR: 0.32394
Epoch[68/20000] Train: DISC | LossD: 0.64766, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.35614 | U_fpR: 0.31791
Epoch[69/20000] Train: DISC | LossD: 0.64594, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.34004 | U_fpR: 0.30584
Epoch[70/20000] Train: DISC | LossD: 0.64558, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.30785 | U_fpR: 0.28370
Epoch[71/20000] Train: DISC | LossD: 0.64360, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.33400 | U_fpR: 0.32193
Epoch[72/20000] Train: DISC | LossD: 0.64349, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.34004 | U_fpR: 0.32193
Epoch[73/20000] Train: DISC

Epoch[128/20000] Train: DISC | LossD: 0.56981, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.36620 | U_fpR: 0.32193
Epoch[129/20000] Train: DISC | LossD: 0.56868, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.35412 | U_fpR: 0.36016
Epoch[130/20000] Train: DISC | LossD: 0.56805, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.32998 | U_fpR: 0.30382
Epoch[131/20000] Train: DISC | LossD: 0.56610, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.36016 | U_fpR: 0.34205
Epoch[132/20000] Train: DISC | LossD: 0.56532, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.26962 | U_fpR: 0.31992
Epoch[133/20000] Train: DISC | LossD: 0.56487, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.33803 | U_fpR: 0.32193
Epoch[134/20000] Train: DISC | LossD: 0.56166, LossG: 94.97743 | Acc: 0.50000 | fpR: 1.00000 | R: 1.00000 | A_fpR: 0.31388 | U_fpR: 0.31590
Epoch[135/20000] Tra

Epoch[187/20000] Train: GEN | LossD: 0.49457, LossG: 85.35185 | Acc: 0.77968 | fpR: 0.44064 | R: 1.00000 | A_fpR: 0.36419 | U_fpR: 0.32394
Epoch[188/20000] Train: GEN | LossD: 0.49457, LossG: 82.61656 | Acc: 0.76157 | fpR: 0.47686 | R: 1.00000 | A_fpR: 0.34004 | U_fpR: 0.37425
Epoch[189/20000] Train: GEN | LossD: 0.49457, LossG: 80.48900 | Acc: 0.76962 | fpR: 0.46076 | R: 1.00000 | A_fpR: 0.31187 | U_fpR: 0.35010
Epoch[190/20000] Train: GEN | LossD: 0.49457, LossG: 77.59480 | Acc: 0.75654 | fpR: 0.48692 | R: 1.00000 | A_fpR: 0.34608 | U_fpR: 0.33602
Epoch[191/20000] Train: GEN | LossD: 0.49457, LossG: 78.06706 | Acc: 0.72938 | fpR: 0.54125 | R: 1.00000 | A_fpR: 0.35815 | U_fpR: 0.36419
Epoch[192/20000] Train: GEN | LossD: 0.49457, LossG: 70.26349 | Acc: 0.73038 | fpR: 0.53924 | R: 1.00000 | A_fpR: 0.33199 | U_fpR: 0.37022
Epoch[193/20000] Train: GEN | LossD: 0.49457, LossG: 71.69301 | Acc: 0.72938 | fpR: 0.54125 | R: 1.00000 | A_fpR: 0.30382 | U_fpR: 0.33199
Epoch[194/20000] Train: GEN

Epoch[254/20000] Train: GEN | LossD: 0.45909, LossG: 44.02280 | Acc: 0.78270 | fpR: 0.43461 | R: 1.00000 | A_fpR: 0.40040 | U_fpR: 0.36217
Epoch[255/20000] Train: GEN | LossD: 0.45909, LossG: 46.06465 | Acc: 0.77465 | fpR: 0.45070 | R: 1.00000 | A_fpR: 0.36419 | U_fpR: 0.35614
Epoch[256/20000] Train: GEN | LossD: 0.45909, LossG: 41.81131 | Acc: 0.77565 | fpR: 0.44869 | R: 1.00000 | A_fpR: 0.35815 | U_fpR: 0.37626
Epoch[257/20000] Train: GEN | LossD: 0.45909, LossG: 43.16729 | Acc: 0.77968 | fpR: 0.44064 | R: 1.00000 | A_fpR: 0.39034 | U_fpR: 0.39034
Epoch[258/20000] Train: GEN | LossD: 0.45909, LossG: 43.52008 | Acc: 0.76459 | fpR: 0.47082 | R: 1.00000 | A_fpR: 0.39638 | U_fpR: 0.40845
Epoch[259/20000] Train: GEN | LossD: 0.45909, LossG: 44.53816 | Acc: 0.76459 | fpR: 0.47082 | R: 1.00000 | A_fpR: 0.37223 | U_fpR: 0.36821
Epoch[260/20000] Train: GEN | LossD: 0.45909, LossG: 43.94131 | Acc: 0.77364 | fpR: 0.45272 | R: 1.00000 | A_fpR: 0.32193 | U_fpR: 0.34608
Epoch[261/20000] Train: GEN

...

Epoch[19734/20000] Train: GEN | LossD: 0.49113, LossG: 1.33551 | Acc: 0.86922 | fpR: 0.01811 | R: 0.75654 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19735/20000] Train: GEN | LossD: 0.49113, LossG: 1.25372 | Acc: 0.85714 | fpR: 0.04225 | R: 0.75654 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19736/20000] Train: GEN | LossD: 0.49113, LossG: 1.17014 | Acc: 0.83400 | fpR: 0.08853 | R: 0.75654 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19737/20000] Train: GEN | LossD: 0.49113, LossG: 1.10220 | Acc: 0.81187 | fpR: 0.13280 | R: 0.75654 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19738/20000] Train: GEN | LossD: 0.49113, LossG: 1.01551 | Acc: 0.77163 | fpR: 0.21328 | R: 0.75654 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19739/20000] Train: GEN | LossD: 0.49113, LossG: 0.93762 | Acc: 0.72334 | fpR: 0.30986 | R: 0.75654 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19740/20000] Train: GEN | LossD: 0.49113, LossG: 0.86685 | Acc: 0.66398 | fpR: 0.42857 | R: 0.75654 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19741/20000] T

Epoch[19798/20000] Train: GEN | LossD: 0.47728, LossG: 1.14492 | Acc: 0.87123 | fpR: 0.03018 | R: 0.77264 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19799/20000] Train: GEN | LossD: 0.47728, LossG: 1.04238 | Acc: 0.84909 | fpR: 0.07445 | R: 0.77264 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19800/20000] Train: GEN | LossD: 0.47728, LossG: 0.98118 | Acc: 0.80181 | fpR: 0.16901 | R: 0.77264 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19801/20000] Train: GEN | LossD: 0.47728, LossG: 0.91946 | Acc: 0.72133 | fpR: 0.32998 | R: 0.77264 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19802/20000] Train: GEN | LossD: 0.47728, LossG: 0.82432 | Acc: 0.60865 | fpR: 0.55533 | R: 0.77264 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19803/20000] Train: GEN | LossD: 0.47728, LossG: 0.77111 | Acc: 0.51308 | fpR: 0.74648 | R: 0.77264 | A_fpR: 1.00000 | U_fpR: 1.00000

Push Generator

Epoch[19804/20000] Train: DISC | LossD: 0.68919, LossG: 0.77111 | Acc: 0.55936 | fpR: 0.65996 | R: 0.77867 | A_fpR: 1.00000 | U_fpR: 1.00000
Ep

Epoch[19856/20000] Train: DISC | LossD: 0.56998, LossG: 0.70812 | Acc: 0.78873 | fpR: 0.05433 | R: 0.63179 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19857/20000] Train: DISC | LossD: 0.55721, LossG: 0.70812 | Acc: 0.79577 | fpR: 0.02616 | R: 0.61771 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19858/20000] Train: DISC | LossD: 0.56666, LossG: 0.70812 | Acc: 0.79980 | fpR: 0.01811 | R: 0.61771 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19859/20000] Train: DISC | LossD: 0.55148, LossG: 0.70812 | Acc: 0.79980 | fpR: 0.01811 | R: 0.61771 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19860/20000] Train: DISC | LossD: 0.54366, LossG: 0.70812 | Acc: 0.80282 | fpR: 0.02213 | R: 0.62777 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19861/20000] Train: DISC | LossD: 0.52903, LossG: 0.70812 | Acc: 0.81388 | fpR: 0.01408 | R: 0.64185 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19862/20000] Train: DISC | LossD: 0.51677, LossG: 0.70812 | Acc: 0.82193 | fpR: 0.01207 | R: 0.65594 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19863/2

Epoch[19913/20000] Train: DISC | LossD: 0.65084, LossG: 0.74791 | Acc: 0.66499 | fpR: 0.43058 | R: 0.76056 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19914/20000] Train: DISC | LossD: 0.64015, LossG: 0.74791 | Acc: 0.68511 | fpR: 0.38833 | R: 0.75855 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19915/20000] Train: DISC | LossD: 0.63430, LossG: 0.74791 | Acc: 0.71529 | fpR: 0.31187 | R: 0.74245 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19916/20000] Train: DISC | LossD: 0.62374, LossG: 0.74791 | Acc: 0.75553 | fpR: 0.22133 | R: 0.73239 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19917/20000] Train: DISC | LossD: 0.59685, LossG: 0.74791 | Acc: 0.80282 | fpR: 0.11066 | R: 0.71630 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19918/20000] Train: DISC | LossD: 0.59610, LossG: 0.74791 | Acc: 0.79879 | fpR: 0.10262 | R: 0.70020 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19919/20000] Train: DISC | LossD: 0.58764, LossG: 0.74791 | Acc: 0.82394 | fpR: 0.04024 | R: 0.68813 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19920/2

Epoch[19973/20000] Train: GEN | LossD: 0.48206, LossG: 1.01916 | Acc: 0.74145 | fpR: 0.26761 | R: 0.75050 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19974/20000] Train: GEN | LossD: 0.48206, LossG: 0.93898 | Acc: 0.72435 | fpR: 0.30181 | R: 0.75050 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19975/20000] Train: GEN | LossD: 0.48206, LossG: 0.87359 | Acc: 0.67002 | fpR: 0.41046 | R: 0.75050 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19976/20000] Train: GEN | LossD: 0.48206, LossG: 0.83660 | Acc: 0.61469 | fpR: 0.52113 | R: 0.75050 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19977/20000] Train: GEN | LossD: 0.48206, LossG: 0.78646 | Acc: 0.55433 | fpR: 0.64185 | R: 0.75050 | A_fpR: 1.00000 | U_fpR: 1.00000
Epoch[19978/20000] Train: GEN | LossD: 0.48206, LossG: 0.71541 | Acc: 0.50905 | fpR: 0.73239 | R: 0.75050 | A_fpR: 1.00000 | U_fpR: 1.00000

Push Generator

Epoch[19979/20000] Train: DISC | LossD: 0.71174, LossG: 0.71541 | Acc: 0.51811 | fpR: 0.72032 | R: 0.75654 | A_fpR: 1.00000 | U_fpR: 1.00000
Ep